In [1]:
%pip install -r requirements.txt
%pwd

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 100.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 57.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 140.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 72.9 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a vers

'/mnt/workspace/my-solution/Spark-TTS'

In [1]:
from http import HTTPStatus
import dashscope
from modelscope import snapshot_download

from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
import os
import subprocess

from tqdm.auto import tqdm
import zipfile

import platform
from cli.SparkTTS import SparkTTS
import torch
import soundfile as sf

2025-08-07 11:16:29.146938: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-07 11:16:29.197101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-07 11:16:30.207500: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# TTS 模型下载

In [2]:
model_dir = snapshot_download('SparkAudio/Spark-TTS-0.5B')

2025-08-07 11:16:35,746 - modelscope - INFO - Target directory already exists, skipping creation.


# 参数设置

In [3]:
# Convert device argument to torch.device
if platform.system() == "Darwin" and torch.backends.mps.is_available():
    # macOS with MPS support (Apple Silicon)
    DEVICE = torch.device("mps:0")
elif torch.cuda.is_available():
    # System with CUDA support
    DEVICE = torch.device("cuda:0")
else:
    # Fall back to CPU
    DEVICE = torch.device("cpu")

DEVICE

device(type='cuda', index=0)

In [4]:
load_dotenv()
API_KEY = os.getenv("DASHSCOPE_API_KEY")
if not API_KEY:
    raise EnvironmentError("DASHSCOPE_API_KEY Not Found")

INPUT_CSV = Path("aigc_speech_generation_tasks/aigc_speech_generation_tasks.csv")
REFERENCE_DIR = INPUT_CSV.parent
MODEL_DIR = model_dir
RESULT_DIR = Path("result")
RESULT_DIR.mkdir(parents=True, exist_ok=True)

tasks = pd.read_csv(INPUT_CSV)
tasks.head()

,utt,reference_speech,text,text_llm
0,1,reference_1.wav,六,六
1,2,reference_2.wav,缴纳罚款单，交警大队开具的违停罚单，编号20240305,缴纳罚款单，交警大队开具的违停罚单，编号二零二四年零三月零五日
2,3,reference_3.wav,腾地而起的风关上了门卷起满屋幽幽檀香味他广袖被风吹起拂过我的眉眼,腾地而起的风关上了门，卷起满屋幽幽檀香味。他广袖被风吹起，拂过我的眉眼。
3,4,reference_4.wav,但这还不是定局一切都将充满变数,但这就不是定局，一切都将充满变数。
4,5,reference_5.wav,从屏风后面转出来看看镜子中的自己，瑞博甚至有种换了个人的感觉,从屏风后面转出来，看看镜子中的自己，瑞博甚至有种换了个人的感觉。


# LLM 前置文本清洗

考虑以下情形：
1. 数字没读好：（2024.12，在不同的场景下读法不一样，例如逐个朗读，还是两千零二十四点一二）价格1280元（十二八） 甲状旁腺激素受体（PTH1R）
2. 标点/符号/单位没读好： mL ℃ +，例如 22 - 24，这里到底是读减号还是读“至/到”，例如 7-9 小时
3. 多音字没读好：例如 奇（鸡音）变偶不变
4. 长段断句：142 199 停不下来
5. 存在空输出 [23, 28, 35, 41, 53, 61, 67, 70, 78， 89, 130，132, 144, 149, 160, 166, 191] (存疑)
6. 没读全/结尾停顿/过长停顿 [84, 90, 94, 97， 136] 例如，94 是一个极长文本，难道是按照句号分隔的么

In [5]:
def call_llm(msg: str, model: str = "qwen-turbo", api_key: str = API_KEY) -> str:
    messages = [{"role": "system", "content": """你是 tts 前置文本清洗专家，回答直接，将以下内容转换为适合语音合成的中文表达，包括：
1. 明确的数字读法；
2. 单位、符号、标点转换；
3. 多音字 disambiguation，用单音字replace多音字；
4. 对没有标点的句子添加适当断句停顿标记（例如逗号、句号、顿号）；

限制：不要随意添加或删减原文不存在的内容，不需要“读作”标识，不需要额外补充信息，你只需要输出转换后的内容。

例如，输入：2024.12是一个重要时间点，那一天我要去兑奖20024.12元，我的彩票编号是20241216，建议每天饮水量为2000-2500 mL，当时是冬天气温应该在20 ℃左右，PTH1R是甲状旁腺激素受体我们都知道1-1=0，奇变偶不变符号看象限。
输出：二零二四年十二月是一个重要的时间点，那一天我要去兑奖两万零二十四点一二元，我的彩票编号是二零二四一二一六。建议每天饮水量为两千到两千五百毫升。当时是冬天，气温应该在二十摄氏度左右。P、T、H、一、R，是甲状旁腺激素受体。我们都知道，一减一等于零。鸡变偶不变，符号看象限。
"""}, {"role": "user", "content": msg}]

    responses = dashscope.Generation.call(
        model=model,
        api_key=api_key,
        messages=messages,
        stream=False,
        result_format='message',  # 将返回结果格式设置为 message
        top_p=0.8,
        temperature=0.6,
        enable_search=False,
        enable_thinking=False,
        thinking_budget=4000
    )

    if responses.status_code == HTTPStatus.OK:
        return responses.output.choices[0]['message']['content']

    raise RuntimeError(f"LLM Error: {responses.message}")

In [6]:
tasks = pd.read_csv(INPUT_CSV)
if "text_llm" not in tasks.columns:
    tasks["text_llm"] = None

for idx, row in tqdm(tasks.iterrows(), total=len(tasks)):
    if pd.isna(row["text_llm"]) or row["text_llm"].strip() == "":
        try:
            cleaned = call_llm(str(row["text"]))
            tasks.at[idx, "text_llm"] = cleaned.strip()
        except Exception as e:
            print(f"[ERROR] LLM failed on utt {row['utt']} with error: {e}")
            tasks.at[idx, "text_llm"] = None

tasks[["utt", "reference_speech", "text", "text_llm"]].to_csv(INPUT_CSV, index=False, encoding="utf-8")

  0%|          | 0/200 [00:00<?, ?it/s]

In [7]:
tasks

,utt,reference_speech,text,text_llm
0,1,reference_1.wav,六,六
1,2,reference_2.wav,缴纳罚款单，交警大队开具的违停罚单，编号20240305,缴纳罚款单，交警大队开具的违停罚单，编号二零二四零三零五。
2,3,reference_3.wav,腾地而起的风关上了门卷起满屋幽幽檀香味他广袖被风吹起拂过我的眉眼,腾地而起的风关上了门，卷起满屋幽幽檀香味。他广袖被风吹起，拂过我的眉眼。
3,4,reference_4.wav,但这还不是定局一切都将充满变数,但這還不是定局，一切將充滿變數。
4,5,reference_5.wav,从屏风后面转出来看看镜子中的自己，瑞博甚至有种换了个人的感觉,从屏风后面转出来，看看镜子中的自己，瑞博甚至有种换了个人的感觉。
...,...,...,...,...
195,196,reference_196.wav,在旅途中那位来自银行的同行告诉我，虽然他是第一次跟我们公司的人一起出差，但他走路的速度一点也...,在旅途中，那位来自银行的同行告诉我，虽然他是第一次跟我们公司的人一起出差，但他的走路速度一点...
196,197,reference_197.wav,未来短视频创作又将走向何方？,未来短视频创作又将走向何方？
197,198,reference_198.wav,虽然他还欠我十块钱没还，但我还是决定再借给他一些，因为他答应下个月一定还清，而且他还一再保证...,虽然他还欠我十块钱没还，但我还是决定再借给他一些，因为他答应下个月一定还清。而且他还一再保证...
198,199,reference_199.wav,伊犁州林业和草原局与伊犁州气象局2023年3月31日17时联合发布森林（草原）火险气象风险橙...,二零二三年三月三十一日十七时，伊犁州林业和草原局与伊犁州气象局联合发布森林（草原）火险气象风...


In [8]:
synthesized_filenames = []
model = SparkTTS(str(MODEL_DIR), DEVICE)

for _, row in tqdm(tasks.iterrows()):
    utt = row["utt"]
    raw_text = str(row["text"])
    text = str(row["text_llm"]) if pd.notna(row["text_llm"]) else raw_text
    ref_audio = REFERENCE_DIR  / row["reference_speech"]
    save_path = RESULT_DIR / "temp.wav"

    print(utt, text, ref_audio)

    with torch.no_grad():
        wav = model.inference(
            text,
            ref_audio,
        )
        sf.write(save_path, wav, samplerate=16000)

    torch.cuda.empty_cache()

    src = save_path
    dst = RESULT_DIR / f"{utt}.wav"
    if dst.exists():
        dst.unlink()
    src.rename(dst)
    synthesized_filenames.append(f"{utt}.wav")

/usr/local/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


0it [00:00, ?it/s]

1 六 aigc_speech_generation_tasks/reference_1.wav


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

198 虽然他还欠我十块钱没还，但我还是决定再借给他一些，因为他答应下个月一定还清。而且他还一再保证，不会让我失望。 aigc_speech_generation_tasks/reference_198.wav


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


199 二零二三年三月三十一日十七时，伊犁州林业和草原局与伊犁州气象局联合发布森林（草原）火险气象风险橙色预警。预计今天夜间至四月九日，伊犁河谷各地以晴间多云天气为主，部分时段有大风天气，空气干燥。正值清明春耕时节，祭祀和烧荒活动较多，极易发生森林草原火灾。伊宁市、霍尔果斯市、霍城县、察布查尔县、伊宁县森林火险气象等级为橙色预警（四级火险）。林牧区、景区加强火源管理，游客禁带火种进入景区。森林草原防火区内禁止烧荒、烧纸、烧香、吸烟玩火和野炊行为。倡导鲜花祭祀、文明祭祀。（预警信息来源：国家预警信息发布中心） aigc_speech_generation_tasks/reference_199.wav


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


200 买体检套餐，基础项目加肿瘤筛查，价格一千二百八十元，用医保支付。 aigc_speech_generation_tasks/reference_200.wav


In [9]:
tasks["synthesized_speech"] = synthesized_filenames
submission_csv = RESULT_DIR / "result.csv"
tasks.to_csv(submission_csv, index=False)

zip_path = Path("./duruii-result.zip")
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in RESULT_DIR.rglob("*"):
        arcname = file_path.relative_to(RESULT_DIR)
        zipf.write(file_path, arcname)

print(f"Submission ready: {zip_path}")

Submission ready: duruii-result.zip
